In [1]:
import os
import glob
import random
import time
import json
import warnings 
import torch.nn.functional as F
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# from utils import label_accuracy_score, add_hist
import cv2

import numpy as np
import pandas as pd
from tqdm import tqdm

# 전처리를 위한 라이브러리
import torchvision
import torchvision.transforms as transforms

#!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2


# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib.patches import Patch
import webcolors
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)

/opt/conda/envs/open-mmlab/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os.path as osp
import mmcv
from mmcv import Config
import pandas as pd
from pandas import DataFrame

import argparse
import os.path as osp
import shutil
import time
import warnings

import mmcv
import torch
from mmcv.cnn.utils import revert_sync_batchnorm
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)
from mmcv.utils import DictAction

from mmseg import digit_version
from mmseg.apis import single_gpu_test
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import setup_multi_processes

In [3]:
os.chdir("../")

In [4]:
## 원하는 config, checkpoint 경로
config_path = '/opt/ml/main/level2-semantic-segmentation-level2-cv-14/mmsegmentation/configs/segformer_test_focal/segformer_test_focal.py'
checkpoint_path = '/opt/ml/main/level2-semantic-segmentation-level2-cv-14/work_dirs/segformer_test_focal/best_mIoU_epoch_54.pth'

In [5]:
# 시각화 palette -> 수정 가능
palette = [
    [0, 0, 0],
    [192, 0, 128],
    [0, 128, 192],
    [0, 128, 64],
    [128, 0, 0], 
    [64, 0, 128], 
    [64, 0, 192],
    [192, 128, 64], 
    [192, 192, 128], 
    [64, 64, 128], 
    [128, 0, 192]
    ]
    
classes = ['Background', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic','Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
category_and_rgb = [[classes, (r,g,b)] for classes, (r, g, b) in zip(classes, palette)]

In [6]:
# DataLoader 생성
cfg = Config.fromfile(config_path)
seed = 42
cfg.data.test.test_mode = True

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

# test loader
test_dataset = build_dataset(cfg.data.test)

2022-05-12 01:46:48,523 - mmseg - INFO - Loaded 624 images


In [ ]:
model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = classes
# model = MMDataParallel(model.cuda(), device_ids=[0])
model.eval()
model = model.cuda()

In [11]:
test_loader = build_dataloader(
        test_dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

In [39]:
pred = []
for sample in tqdm(test_loader):
    logit = model.whole_inference(img=sample['img'][0].cuda(), img_meta=sample['img_metas'][0], rescale=False)
    pred.extend(logit.detach().cpu().numpy())

100%|██████████| 624/624 [00:40<00:00, 15.39it/s]


In [45]:
pred = np.array(pred)

In [46]:
import joblib
joblib.dump(pred, 'save.pkl')